# Importing Libraries

In [ ]:
import os

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_together import ChatTogether

from dotenv import load_dotenv

load_dotenv()

TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")

together_model_name = "Qwen/Qwen3-235B-A22B-Instruct-2507-tput"

llm = ChatTogether(
    together_api_key=TOGETHER_API_KEY,
    model=together_model_name,
)

### Testing

In [5]:
res = llm.invoke("How is the weather in Bengaluru today?")
print(res.content)

I can't provide real-time information such as today's weather. For the most accurate and up-to-date weather in Bengaluru, I recommend checking a trusted weather website or app like the BBC Weather, AccuWeather, or the Weather Channel.


# Defining Tools

In [6]:
from langchain.tools import tool

@tool

def get_weather(location: str):
    """
    Call to get the current weather
    """

    if location.lower() in ['bengaluru', "mysuru"]:
        return "It is 23 degrees Celsius and cloudy."
    else:
        return "It is 25 degree Celsius and sunny."
    

@tool
def check_seating_availability(location: str, seating_type: str):
    """
    Call to check seating availability
    """

    if location.lower() == "bengaluru" and seating_type.lower()=="outdoor":
        return "Yes, we still have seats available outdoors."
    elif location.lower() == "bengaluru" and seating_type.lower() == "indoor":
        return "Yes, we have indoor seating available."
    else:
        return "Sorry, seating information for this location is unavailable."
    
tools = [get_weather, check_seating_availability]

In [7]:
llm_with_tools = llm.bind_tools(tools)

result = llm_with_tools.invoke("How will the weather be in bengaluru today.")
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_goo2ne3e5orvdl3x5dr0zfmm', 'function': {'arguments': '{"location":"bengaluru"}', 'name': 'get_weather'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 232, 'total_tokens': 255, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-235B-A22B-Instruct-2507-tput', 'system_fingerprint': None, 'id': 'o8GvnVN-zqrih-9738f091eef1c1b3', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--b3bbdfb2-d20a-4442-92bf-d774195100ef-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'bengaluru'}, 'id': 'call_goo2ne3e5orvdl3x5dr0zfmm', 'type': 'tool_call'}], usage_metadata={'input_tokens': 232, 'output_tokens': 23, 'total_tokens': 255, 'input_token_details': {}, 'output_token_details': {}})

In [8]:
print(result.content)

In [9]:
result.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'bengaluru'},
  'id': 'call_goo2ne3e5orvdl3x5dr0zfmm',
  'type': 'tool_call'}]

# Setting up messages

In [10]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [
    HumanMessage("How will the weather be in bengaluru today? Do you still have seats outdoor available?")
]

llm_output = llm_with_tools.invoke(messages)
llm_output

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_cgbtwrp25w20qfywamzulm5z', 'function': {'arguments': '{"location":"bengaluru"}', 'name': 'get_weather'}, 'type': 'function', 'index': 0}, {'id': 'call_eijnl63van8wnr6cgry93ni9', 'function': {'arguments': '{"location":"bengaluru","seating_type":"outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function', 'index': 1}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 240, 'total_tokens': 298, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-235B-A22B-Instruct-2507-tput', 'system_fingerprint': None, 'id': 'o8GvpsP-28Eivz-9738f0c5b8dbc174', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--48e22a7f-cd49-4b4a-ab4f-aa04a7f50f3d-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'bengaluru'}, 'id': 'call_cgbtwrp25w20qfywamzulm5z', 'type': 'tool_call'}, {'name': 'check_seating_availabil

In [11]:
print(llm_output.content)

### Tools getting called by LLM

In [12]:
llm_output.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'bengaluru'},
  'id': 'call_cgbtwrp25w20qfywamzulm5z',
  'type': 'tool_call'},
 {'name': 'check_seating_availability',
  'args': {'location': 'bengaluru', 'seating_type': 'outdoor'},
  'id': 'call_eijnl63van8wnr6cgry93ni9',
  'type': 'tool_call'}]

In [13]:
messages.append(llm_output)
messages

[HumanMessage(content='How will the weather be in bengaluru today? Do you still have seats outdoor available?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_cgbtwrp25w20qfywamzulm5z', 'function': {'arguments': '{"location":"bengaluru"}', 'name': 'get_weather'}, 'type': 'function', 'index': 0}, {'id': 'call_eijnl63van8wnr6cgry93ni9', 'function': {'arguments': '{"location":"bengaluru","seating_type":"outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function', 'index': 1}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 240, 'total_tokens': 298, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-235B-A22B-Instruct-2507-tput', 'system_fingerprint': None, 'id': 'o8GvpsP-28Eivz-9738f0c5b8dbc174', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--48e22a7f-cd49-4b4a-ab4f-aa04a7f50f3d-0', tool_ca

In [14]:
tool_mapping = {
    "get_weather": get_weather,
    "check_seating_availability": check_seating_availability
}

In [15]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call['args'])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call['id']))

In [16]:
messages

[HumanMessage(content='How will the weather be in bengaluru today? Do you still have seats outdoor available?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_cgbtwrp25w20qfywamzulm5z', 'function': {'arguments': '{"location":"bengaluru"}', 'name': 'get_weather'}, 'type': 'function', 'index': 0}, {'id': 'call_eijnl63van8wnr6cgry93ni9', 'function': {'arguments': '{"location":"bengaluru","seating_type":"outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function', 'index': 1}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 240, 'total_tokens': 298, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-235B-A22B-Instruct-2507-tput', 'system_fingerprint': None, 'id': 'o8GvpsP-28Eivz-9738f0c5b8dbc174', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--48e22a7f-cd49-4b4a-ab4f-aa04a7f50f3d-0', tool_ca

In [17]:
res = llm_with_tools.invoke(messages)
res

AIMessage(content='The weather in Bengaluru today is 23 degrees Celsius and cloudy. Good news — there are still outdoor seats available!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 333, 'total_tokens': 359, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-235B-A22B-Instruct-2507-tput', 'system_fingerprint': None, 'id': 'o8Gvtup-zqrih-9738f11a88a48839', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--7805f7c4-29d7-41ec-a598-71be626e928e-0', usage_metadata={'input_tokens': 333, 'output_tokens': 26, 'total_tokens': 359, 'input_token_details': {}, 'output_token_details': {}})

In [19]:
print(res.content)

The weather in Bengaluru today is 23 degrees Celsius and cloudy. Good news — there are still outdoor seats available!
